#### DATA

In [1]:
import os
import json
import pandas as pd # Gzip 압축된 CSV 파일 불러오기

df = pd.read_csv("data.csv.gz", compression='gzip')
df.head()


,standard_form,dialect_form
0,난 진짜 벌초 할때 뱀나오잖아 뱀도,난 진짜 벌초 할때 뱀나오잖아 뱀도
1,어어 그니까,어어 그니까
2,아 잘도,아 잘도
3,예초기 뱀도 짤려,예초기 뱀도 짤려
4,어 그니까 완전 끔찍해,어 그니까 완전 끔찍해


In [2]:
# 중복 제거
df_filtered = df[df["standard_form"] != df["dialect_form"]]
df_filtered = df_filtered.drop_duplicates()
df_filtered.info()

<class 'pandas.core.frame.DataFrame'>
Index: 1116389 entries, 6 to 2774254
Data columns (total 2 columns):
 #   Column         Non-Null Count    Dtype 
---  ------         --------------    ----- 
 0   standard_form  1116389 non-null  object
 1   dialect_form   1116389 non-null  object
dtypes: object(2)
memory usage: 25.6+ MB


In [3]:
df_filtered.reset_index(drop=True, inplace=True)
df_filtered.head()

,standard_form,dialect_form
0,예전에 그렇게,예전에 겅
1,아 그러니까 예초 할땐 조심해야 돼,아 겅하니까이 예초 할땐 조심해야 돼
2,그러고 예초 벌초(하다그네)/(하다가),겅허고 예초 벌초(하다그네)/(하다가)
3,어 그거 튀겨서,어 그거 튀겨그네
4,돌멩이 해 가지고 실명된사람 있어,돌멩이 해 가지고 실명된사람 이서


In [3]:
# df_sampled = df_filtered.sample(frac=0.1, random_state=42)  # Adjust the fraction as needed
# df_sampled.info()

<class 'pandas.core.frame.DataFrame'>
Index: 111639 entries, 1441681 to 2360878
Data columns (total 2 columns):
 #   Column         Non-Null Count   Dtype 
---  ------         --------------   ----- 
 0   standard_form  111639 non-null  object
 1   dialect_form   111639 non-null  object
dtypes: object(2)
memory usage: 2.6+ MB


In [3]:
import sentencepiece as spm
from tokenizers import Tokenizer, normalizers
from tokenizers.models import BPE
from tokenizers.pre_tokenizers import Whitespace
from tokenizers.trainers import BpeTrainer
from tokenizers.processors import TemplateProcessing
from datasets import Dataset
from transformers import PreTrainedTokenizerFast, AutoModelForSeq2SeqLM, Seq2SeqTrainer, Seq2SeqTrainingArguments


class CustomSPTokenizer(PreTrainedTokenizerFast):
    def __init__(self, tokenizer_path):
        super().__init__(tokenizer_file=tokenizer_path)
        self._tokenizer = Tokenizer.from_file(tokenizer_path)

    def _tokenize(self, text):
        return self._tokenizer.encode(text).tokens

    def _convert_token_to_id(self, token):
        return self._tokenizer.token_to_id(token)

    def _convert_id_to_token(self, index):
        return self._tokenizer.id_to_token(index)

    def convert_tokens_to_string(self, tokens):
        return self._tokenizer.decode(tokens)

    def build_inputs_with_special_tokens(self, token_ids_0, token_ids_1=None):
        if token_ids_1 is None:
            return [self.cls_token_id] + token_ids_0 + [self.sep_token_id]
        cls = [self.cls_token_id]
        sep = [self.sep_token_id]
        return cls + token_ids_0 + sep + token_ids_1 + sep + [self.sep_token_id]


    @property
    def cls_token_id(self):
        return self._tokenizer.token_to_id("<cls>")

    @property
    def sep_token_id(self):
        return self._tokenizer.token_to_id("<sep>")

    @property
    def pad_token_id(self):
        return self._tokenizer.token_to_id("<pad>")

    @property
    def unk_token_id(self):
        return self._tokenizer.token_to_id("<unk>")



tokenizer_path = "bpe_tokenizer.json"
custom_tokenizer = CustomSPTokenizer(tokenizer_path)

custom_tokenizer.add_special_tokens({'pad_token': '<pad>'})
custom_tokenizer.pad_token = '<pad>'

print("Padding token ID:", custom_tokenizer.pad_token_id)

Padding token ID: 1


In [5]:
from sklearn.model_selection import train_test_split
train_df, eval_df = train_test_split(df_filtered, test_size=0.2, random_state=42)   ## 89311, 22328


train_dataset = Dataset.from_pandas(train_df)
eval_dataset = Dataset.from_pandas(eval_df)


def preprocess_function(examples):
    tokenized_data = []
    inputs = [ex for ex in examples['dialect_form']]  ## 사투리
    targets = [ex for ex in examples['standard_form']]  ## 표준어 
    model_inputs = custom_tokenizer(inputs, max_length=100, truncation=True, padding="max_length")

    
    with custom_tokenizer.as_target_tokenizer():
        labels = custom_tokenizer(targets, max_length=100, truncation=True, padding="max_length")
    
    
    model_inputs["labels"] = labels["input_ids"]
    return model_inputs

train_dataset = train_dataset.map(preprocess_function, batched=True)
eval_dataset = eval_dataset.map(preprocess_function, batched=True)


Map:   0%|          | 0/893111 [00:00<?, ? examples/s]

c:\Users\AI06\anaconda3\envs\nlp\lib\site-packages\transformers\tokenization_utils_base.py:3921: UserWarning: `as_target_tokenizer` is deprecated and will be removed in v5 of Transformers. You can tokenize your labels by using the argument `text_target` of the regular `__call__` method (either in the same call as your input texts if you use the same keyword arguments, or in a separate call.
  warnings.warn(


Map:   0%|          | 0/223278 [00:00<?, ? examples/s]

In [7]:
train_dataset = train_dataset.remove_columns(['standard_form', 'dialect_form', '__index_level_0__', 'token_type_ids'])
eval_dataset = eval_dataset.remove_columns(['standard_form', 'dialect_form', '__index_level_0__', 'token_type_ids'])
print(train_dataset.column_names)  
print(eval_dataset.column_names) 

['input_ids', 'attention_mask', 'labels']
['input_ids', 'attention_mask', 'labels']


In [ ]:
import torch 
import os
import pandas as pd
from transformers import BertTokenizer, AutoTokenizer, AutoModelForSeq2SeqLM, Seq2SeqTrainer, Seq2SeqTrainingArguments
from datasets import Dataset


device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
print(f"Using device: {device}")


model_name = "gogamza/kobart-base-v2"
tokenizer = AutoTokenizer.from_pretrained(model_name)
model = AutoModelForSeq2SeqLM.from_pretrained(model_name)


device = "cuda" if torch.cuda.is_available() else "cpu"
print(device)
model.to(device)


# model = AutoModelForSeq2SeqLM.from_pretrained("facebook/bart-base")
# model.resize_token_embeddings(len(custom_tokenizer))

In [21]:
training_args = Seq2SeqTrainingArguments(
    output_dir="./results2",
    evaluation_strategy="steps",  # 'epoch' 대신 'steps'로 설정하여 주기적으로 평가
    eval_steps=500,  # 평가 주기를 500 스텝으로 설정
    learning_rate=2e-5,  # 일반적으로 2e-5와 같은 낮은 학습률이 안정적
    per_device_train_batch_size=32,  # GPU 메모리와 성능에 따라 조정
    per_device_eval_batch_size=32,  # 평가 시에도 배치 크기를 맞춤
    num_train_epochs=1,  # 기본 에포크 수
    weight_decay=0.01,  # 가중치 감쇠
    logging_dir="./logs",  # 로그 디렉토리
    logging_steps=100,  # 로그 주기를 100 스텝으로 설정
    save_total_limit=2,  # 저장할 체크포인트 수 제한
    load_best_model_at_end=True,  # 학습이 끝난 후 가장 좋은 모델을 로드
    gradient_accumulation_steps=2,  # 그래디언트 누적을 통해 배치 크기 증가 효과
)

# Trainer
trainer = Seq2SeqTrainer(
    model=model,
    args=training_args,
    train_dataset=train_dataset,
    eval_dataset=eval_dataset,
    tokenizer=custom_tokenizer,
)

In [ ]:
trainer.train()

# 모델 저장
model.save_pretrained("./bartbase_model2")

In [15]:
# def remove_padding(tokens, attention_mask):
#     # 패딩이 시작되는 지점(첫 번째 0이 나타나는 위치)를 찾음
#     non_padded_length = attention_mask.tolist().index(0) if 0 in attention_mask else len(attention_mask)
#     # 패딩이 시작되는 지점까지 자름
#     trimmed_tokens = tokens[:non_padded_length]
#     return trimmed_tokens

def translate(text):
    # 입력 텍스트를 토크나이징
    inputs = custom_tokenizer(text, return_tensors="pt")
    inputs = {key: value.to(device) for key, value in inputs.items() if key != 'token_type_ids'}
    print(inputs)
    print(len(inputs['input_ids'][0]))
    
    # 예측 생성
    outputs = model.generate(
        input_ids=inputs['input_ids'],
        attention_mask=inputs['attention_mask'],
        max_length=100,
    )
    
    # 생성된 토큰을 텍스트로 디코딩하기 전에 패딩 제거
    # output_ids = remove_padding(outputs[0], outputs[0].ne(custom_tokenizer.pad_token_id).int())
    translated_text = custom_tokenizer.decode(outputs[0], skip_special_tokens=True)
    # print(f"Trimmed Output IDs: {translated_text}")
    
    return translated_text

# 예제 번역
sample_sentence = "아방 어디 간?"
translated_sentence = translate(sample_sentence)
print(f"Standard: {sample_sentence}")
print(f"Dialect: {translated_sentence}")

{'input_ids': tensor([[   2, 3336, 3252,  132,   35,    3]], device='cuda:0'), 'attention_mask': tensor([[1, 1, 1, 1, 1, 1]], device='cuda:0')}
6
Standard: 아방 어디 간?
Dialect: 아빠 어디 갔어?


#### 이어서 학습 

In [1]:
import pandas as pd

# http://www.jejuda114.com/%EC%9E%AC%EB%B0%8C%EB%8A%94-%EC%A0%9C%EC%A3%BC-%EB%B0%A9%EC%96%B8%EC%9D%84-%EB%B0%B0%EC%9B%8C%EB%B3%BC%EA%B9%8C%EC%9A%94/

text = """
혼저옵서 – 어서오세요 / 옵데강, 혼저오십서 – 오셨습니까, 어서 오십시오.
왕 봥 갑서 – 와서 보고 가세요 / 혼저 혼저, 재게 재게 옵서 – 어서 어서, 빨리빨리 오세요
도르멍 도르멍 옵서 – 뛰면서 오세요 / 놀멍 놀멍 봅서 – 천천히 보세요 / 이시냐 – 있느냐
또시 꼭 옵서양 – 다시 꼭 오세요 / 하영봅서 – 많이 보세요 / 강옵서 – 갔다오세요
쉬영갑서 – 쉬어서 가세요 / 햄시냐 – 하고있느냐 / 와시냐 – 왔느냐
검절매레 안갈꺼과 – 김매러 안가실겁니까 / 좋쑤과 – 좋습니까 / 이쑤과 – 있습니까.
알았수다 – 알았습니다 / 침대방 허쿠과. 온돌방 쓰쿠과 – 침대방 하겠어요? 온돌방 쓰겠어요.
여기서 서울더레 해집주양? – 여기서 서울에 전화할 수 있지요? / 어디서 옵데가? – 어디서 오셨습니까.
제주엔 참 종거 만쑤다양 – 제주엔 참 좋은 것이 많이 있습니다. / 날봅서. 있쑤과 – 여보세요. 계십니까?
저녁에랑 전복죽 쒀줍서양 – 저녁 식사 때는 전복죽을 쑤어주세요. / 예. 어서 옵써 – 예 어서 오십시요.
얼마나 사쿠꽈 – 얼마나 사겠습니까. / 많이팝서 다시 오쿠다 양 – 많이 파십시오, 다시 오겠습니다.
영 갑서양 – 이쪽으로 가십시오. / 이거 얼마우꽈 – 이거 얼마입니까. / 오라시냐 – 오라고 하느냐?
왕갈랑갑서 – 와서 나누어 가지고 가세요. / 경허지 맙서 – 그렇게 하지 마세요 / 가시냐 – 갔느냐?
차탕갑서 – 차를 타고 가세요. / 놀암시냐 – 놀고 있느냐? / 감시냐 – 가느냐?
저디 산 것 말이우꽈? – 저기 서 있는 것 말입니까? / 이거 얼마우꽈 – 이거 얼마입니까.
어디 감수꽈 – 어디 가십니까? / 낼 오쿠꽈 – 내일 오시겠습니까? / 골암수꽈 – 말하십니까?
맛조수다게 – 맛있습니다. / 어느제 오쿠과 – 언제 오시겠습니까? / 놀당 갑서양 – 놀다가 가십시요.
도르멍 도르멍 가다 – 뛰면서 뛰어가다. / 볕이 과랑과랑 허다 – 햇볕이 쨍쨍나다.
폭싹 속았수다 – 매우 수고하셨습니다
혼저 옵서. 제주도 사투리로 말 호난 – 어서 오십시오. 제주도 사투리로 말 하니까.
무신 거옌 고람 신디 몰르쿠게? – 뭐라고 말하는지 모르겠지요?
게메 마씀, 귀 눈이 왁왁하우다 – 글세 말입니다. 귀와 눈이 캄캄합니다.
경해도 고만히 생각호멍 들으민 조금씩 알아집니다 – 그래도 가만히 생각하며 들어면 조금씩 알게 됩니다.
제주도 사투리 촘말로 귀하고 아름다운 보물이우다 – 제주도 사투리 정말로 귀하고 아름다운 보물입니다.
펜안 하우꽈? 제주도엔 오난 어떵 하우꽈? – 편안하십니까? 제주도에 오니 어떠하십니까.
촘말로 좋수다. 공기도 맑고 – 정말로 좋습니다. 공기도 맑고,
산이영 바당이영 몬딱 좋은게 마씀 – 산이랑 바다랑 모두가 좋습니다.
서울에 갈 때랑 하영 담앙 갑서 – 서울에 갈 때는 많이 담아서 가십시오.
게메, 양. 경 해시민 얼마나 좋코 마씀? – 그러게 말입니다. 그렇게 했으면 얼마나 좋겠습니까?
저기, 물허벅 정 가는 거, 비바리덜 아니꽈? – 저기, 물허벅(바구니)지고 가는 거, 처녀들 아닙니까?
맞수다. 비바리도 있고, 넹바리도 있수다 – 맞습니다. 처녀도 있고, 시집 간 여자도 있습니다.
비바리덜 곱들락 호고 놀씬하우다 양! – 처녀들도 곱고 날씬하군요!
안아 보곡 소랑호젠 – 안아보고 사랑할려고 / 호꼼만 이십서게 – 조금만 계십시오.
몽케지 마랑 혼저 오라게 – 꾸물대지 말고 어서 오너라 / 이 조끄뜨레 오라게 – 여기 가까이(옆에) 오너라.
무사 조끄뜨레만 오랜 햄수꽈? – 왜 가까이(옆에)만 오라고 하십니까?
호꼼이라도 고치만 있구정 호연 – 조금이라도 같이만 잇고 싶어서 / 놈덜 우습니다 – 남들이 웃습니다.
어떵 호느냐? 소랑에는 부치름이 엇나 – 어떠하느냐? 사랑네는 부끄러움이 없단다.
조끄뜨레 하기엔 하영멍 당신 – 가까이 하기엔 머나먼 당신 / 혼저 왕 먹읍서 – 어서 와서 먹으십시오.
맨도롱 하우꽈? – 따뜻합니까? / 똣똣혼 게 먹기 똑 좋았수다 – 따끈따끈한 것이 먹기에 꼭 좋았습니다.
맨도롱 홀 때 호로록 들여 싸붑서 – 따뜻할 때 후루룩 마셔 버리십시오.
과랑 과랑혼 벳디 – 쨍쨍한 햇볕속에 / 일 호젠 호난 속았수다 – 일 하려고 하니 수고 했습니다.
속을 거 있수과? 호꼼, 똠은 났수다만 – 수고 할 거 있습니까? 조금 땀은 났습니다만,
안트레 들어 왕, 저녁 먹엉 갑서 – 안으로 들어 오셔서, 저녁식사 하고 가십시오.기본 단어
아 방 – 아버지 / 어 멍 – 어머니 / 하르방 – 할아버지 / 할 망 – 할머니 / 다 슴 – 의붓
다슴아덜 – 의붓아들 / 다슴똘 – 다슴딸 / 씨아방 – 시아버지 / 씨어멍 – 시어머니
씨아주방 – 남편의 형제 / 족은아덜 – 작은 아들 / 메누리 – 며누리 / 가시아방 – 장인
가시어멍 – 장모 / 동세 – 동서 / 예펜 – 여편네 / 소나이 – 사나이
제집아이 – 여자아이 / 비바리 – 처녀 / 두가시 – 부부 / 두린아이 – 어린아이
괜당 – 친족 / 사농바치 – 사냥꾼 / 꿩바치 – 꿩사냥꾼 / 아시 – 동생
아저씨 – 아주방 / 아주머니 – 아주망 / 말젯놈 – 세쨋놈동물
고냥 독생이 – 굴뚝새 / 놀개기 – 날개 / 박생이 – 동박새 / 독 – 닭 / 고냉이 – 고양이
송애기 – 송아지 / 몽생이 – 망아지 / 돗 – 돼지 / 부랭이 – 수소 / 염송애기 – 염소
강생이 – 강아지 / 중이 – 쥐 / 갓돔 – 도미 / 겡이 – 게 / 구젱기 – 소라 / 물꾸럭 – 문어
게염지 – 개미 / 멩마구리 – 맹꽁이 / 베랭이 – 버러지 / 베염 – 뱀 / 빙애기 – 병아리
생이 – 새 / 나람쥐 – 다람쥐 / 쥉이 – 쥐 / 여호 – 여우 / 노리 – 노루 / 빙아리 – 병아리
쉐 – 소 / 쉐막 – 외양간 / 다세기 – 돼지 / 두테비 – 두꺼비 / 골개비 – 개구리
감저 – 고구마 / 지실 – 감자 / 모물 – 메밀 / 어욱 – 억새 / 태역 – 잔디
재배낭 – 구실잣밤나무 / 굴묵낭 – 느티나무 / 가라지 – 강아지풀 / 숙대낭 – 삼나무
대죽 – 수수 / 콥대사니 – 마늘 / 마농 – 파 / 꿩마농 – 달래 / 각매기 – 빈 이삭
고고리 – 이삭 / 콩각매기 – 콩깍지 / 놈삐 – 무 / 노몰 – 채소 / 베체기 – 질경이
하르비고장 – 할미꽃 / 소앵이 – 엉겅퀴 / 천상쿨 – 망초, 개망초 / 낭께기 – 나무가지
낭꺼죽 – 나무 껍질 / 곡석 – 곡식 / 콩꼬질 – 빈 콩깍지, 빈 이삭 / 촐 – 꼴
가시자왈 – 가시덤불 / 쇠우리 – 부추

"""


# 텍스트를 줄 단위로 분리
lines = text.strip().split('\n')

# 방언과 표준어를 저장할 리스트
dialect_form = []
standard_form = []

# 각 줄을 '/'로 분리하고 방언과 표준어를 리스트에 추가
for line in lines:
    pairs = line.split(' / ')
    for pair in pairs:
        dialect, standard = pair.split(' – ')
        dialect_form.append(dialect.strip())
        standard_form.append(standard.strip())

# 데이터프레임 생성
df = pd.DataFrame({
    'dialect_form': dialect_form,
    'standard_form': standard_form
})

df.head()

,dialect_form,standard_form
0,혼저옵서,어서오세요
1,"옵데강, 혼저오십서","오셨습니까, 어서 오십시오."
2,왕 봥 갑서,와서 보고 가세요
3,"혼저 혼저, 재게 재게 옵서","어서 어서, 빨리빨리 오세요"
4,도르멍 도르멍 옵서,뛰면서 오세요
